In [ ]:
import math

def expand_poly_mod(a, n, mod):
    """Computes (x + a)^n mod (x^r - 1, n)"""
    result = [1]

    bin_n = bin(n)[2:]

    poly = [a, 1]

    for bit in bin_n:
        result = multiply_poly(result, result, mod)

        if bit == '1':
            result = multiply_poly(result, poly, mod)

    return result

def multiply_poly(poly1, poly2, mod):
    """Multiply two polynomials modulo mod"""
    result = [0] * (len(poly1) + len(poly2) - 1)

    for i in range(len(poly1)):
        for j in range(len(poly2)):
            result[i + j] = (result[i + j] + poly1[i] * poly2[j]) % mod

    return result

def reduce_poly_mod_xr_minus_1(poly, r, mod):
    """Reduce polynomial modulo (x^r - 1, n)"""
    result = [0] * r

    for i in range(len(poly)):
        result[i % r] = (result[i % r] + poly[i]) % mod

    return result

def aks_primality_test(n):
    """
    AKS primality test implementation
    """
    max_log = int(math.log2(n)) + 1
    for b in range(2, max_log + 1):
        a = int(n ** (1 / b))
        if a ** b == n:
            return False

    log_squared = (math.log2(n)) ** 2
    r = 2

    while r <= max(3, log_squared):
        if math.gcd(n, r) > 1:
            if n > r:
                return False
            return n == r

        ord_r = 1
        power = n % r

        while power != 1 and ord_r <= log_squared:
            power = (power * n) % r
            ord_r += 1

        if ord_r > log_squared:
            break

        r += 1

    for a in range(2, r + 1):
        if math.gcd(a, n) > 1:
            return a == n

    if n <= r:
        return True

    limit = int(math.sqrt(r - 1) * math.log(n))

    for a in range(1, limit + 1):
        poly = [a, 1]
        left_poly = expand_poly_mod(a, n, n)
        left_poly = reduce_poly_mod_xr_minus_1(left_poly, r, n)

        right_poly = [0] * r
        right_poly[0] = a
        right_poly[n % r] = (right_poly[n % r] + 1) % n

        if left_poly != right_poly:
            return False

    return True

if __name__ == "__main__":
    test_numbers = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37]

    print("Testing AKS primality test:")
    for n in test_numbers:
        result = aks_primality_test(n)
        print(f"{n} is prime: {result}")

    print("\nTesting some composites:")
    composites = [4, 6, 8, 9, 10, 12, 14, 15, 16, 18, 20]
    for n in composites:
        result = aks_primality_test(n)
        print(f"{n} is prime: {result}")

Testing AKS primality test:
2 is prime: True
3 is prime: True
5 is prime: True
7 is prime: True
11 is prime: True
13 is prime: True
17 is prime: True
19 is prime: True
23 is prime: True
29 is prime: True
31 is prime: True
37 is prime: True

Testing some composites:
4 is prime: False
6 is prime: False
8 is prime: False
9 is prime: False
10 is prime: False
12 is prime: False
14 is prime: False
15 is prime: False
16 is prime: False
18 is prime: False
20 is prime: False


In [ ]:
import random

def power_mod(base, exponent, modulus):
    """Compute (base^exponent) % modulus efficiently using square-and-multiply"""
    result = 1
    base = base % modulus
    while exponent > 0:
        if exponent % 2 == 1:
            result = (result * base) % modulus
        exponent = exponent >> 1
        base = (base * base) % modulus
    return result

def miller_rabin_test(n, k=40):
    """
    Miller-Rabin primality test
    n: number to test for primality
    k: number of rounds/witnesses to test
    Returns: True if probably prime, False if definitely composite
    """
    if n == 2 or n == 3:
        return True
    if n <= 1 or n % 2 == 0:
        return False

    r, d = 0, n - 1
    while d % 2 == 0:
        r += 1
        d //= 2

    for _ in range(k):
        a = random.randint(2, n - 2)
        x = power_mod(a, d, n)

        if x == 1 or x == n - 1:
            continue

        for _ in range(r - 1):
            x = power_mod(x, 2, n)
            if x == n - 1:
                break
        else:
            return False

    return True

def is_probably_prime(n, k=40):
    """
    Wrapper function to handle large integers
    n: number to test (can be integer or string representation)
    k: number of witnesses to test
    """
    if isinstance(n, str):
        n = int(n)
    return miller_rabin_test(n, k)

def generate_large_prime(bits, k=40):
    """
    Generate a random prime number with the specified bit length
    bits: number of bits (e.g., 1024 for a 1024-bit prime)
    k: number of witnesses for Miller-Rabin test
    """
    while True:
        p = random.getrandbits(bits) | (1 << (bits - 1)) | 1

        if is_probably_prime(p, k):
            return p

if __name__ == "__main__":
    test_numbers = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37]

    print("Testing Miller-Rabin primality test:")
    for n in test_numbers:
        result = is_probably_prime(n)
        print(f"{n} is prime: {result}")

    print("\nTesting some composites:")
    composites = [4, 6, 8, 9, 10, 12, 14, 15, 16, 18, 20]
    for n in composites:
        result = is_probably_prime(n)
        print(f"{n} is prime: {result}")

    print("\nGenerating a 512-bit prime:")
    prime = generate_large_prime(512)
    print(f"Generated prime: {prime}")
    print(f"Number of digits: {len(str(prime))}")
    print(f"Verified prime by Rabin's Algo: {is_probably_prime(prime)}")

Testing Miller-Rabin primality test:
2 is prime: True
3 is prime: True
5 is prime: True
7 is prime: True
11 is prime: True
13 is prime: True
17 is prime: True
19 is prime: True
23 is prime: True
29 is prime: True
31 is prime: True
37 is prime: True

Testing some composites:
4 is prime: False
6 is prime: False
8 is prime: False
9 is prime: False
10 is prime: False
12 is prime: False
14 is prime: False
15 is prime: False
16 is prime: False
18 is prime: False
20 is prime: False

Generating a 512-bit prime:
Generated prime: 10853048568003711068766891070920972061928459173696653540222503660567040663605454454007938069503892145356756749537989121401953111822250468986559355138601009
Number of digits: 155
Verified prime by Rabin's Algo: True


In [ ]:
import time
import math

def run_comparison_test(max_n=100):
    """
    Compare AKS and Miller-Rabin algorithms for small numbers
    """
    print("Comparing AKS and Miller-Rabin for primes up to", max_n)
    print("Number | AKS Result | AKS Time | M-R Result | M-R Time")
    print("-" * 60)

    for n in range(2, max_n + 1):
        start_time = time.time()
        try:
            aks_result = aks_primality_test(n)
            aks_time = time.time() - start_time
            aks_status = str(aks_result)
        except Exception as e:
            aks_time = time.time() - start_time
            aks_status = "ERROR"

        start_time = time.time()
        mr_result = is_probably_prime(n)
        mr_time = time.time() - start_time

        print(f"{n:6d} | {aks_status:10s} | {aks_time:.6f}s | {str(mr_result):10s} | {mr_time:.6f}s")

def compare_specific_numbers():
    """
    Compare performance on specific test numbers
    """
    test_numbers = [11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71]

    print("\nComparing AKS and Miller-Rabin for specific primes:")
    print("Number | AKS Result | AKS Time | M-R Result | M-R Time")
    print("-" * 60)

    for n in test_numbers:
        start_time = time.time()
        aks_result = aks_primality_test(n)
        aks_time = time.time() - start_time

        start_time = time.time()
        mr_result = is_probably_prime(n)
        mr_time = time.time() - start_time

        print(f"{n:6d} | {str(aks_result):10s} | {aks_time:.6f}s | {str(mr_result):10s} | {mr_time:.6f}s")

def compare_medium_primes():
    """
    Compare performance on medium-sized primes where AKS is still feasible
    """
    medium_primes = [101, 211, 307, 401, 503, 601, 701, 809, 907, 1009]

    print("\nComparing AKS and Miller-Rabin for medium-sized primes:")
    print("Number | AKS Result | AKS Time | M-R Result | M-R Time | Time Ratio")
    print("-" * 75)

    for n in medium_primes:
        start_time = time.time()
        try:
            aks_result = aks_primality_test(n)
            aks_time = time.time() - start_time
            aks_status = str(aks_result)
        except Exception as e:
            aks_time = time.time() - start_time
            aks_status = "ERROR"

        start_time = time.time()
        mr_result = is_probably_prime(n)
        mr_time = time.time() - start_time

        if mr_time > 0:
            time_ratio = aks_time / mr_time
        else:
            time_ratio = float('inf')

        print(f"{n:6d} | {aks_status:10s} | {aks_time:.6f}s | {str(mr_result):10s} | {mr_time:.6f}s | {time_ratio:.2f}x")

if __name__ == "__main__":
    print("Running algorithm comparison tests...")

    run_comparison_test(50)

    compare_specific_numbers()

    compare_medium_primes()

Running algorithm comparison tests...
Comparing AKS and Miller-Rabin for primes up to 50
Number | AKS Result | AKS Time | M-R Result | M-R Time
------------------------------------------------------------
     2 | True       | 0.000012s | True       | 0.000002s
     3 | True       | 0.000006s | True       | 0.000001s
     4 | False      | 0.000002s | False      | 0.000001s
     5 | True       | 0.000007s | True       | 0.000111s
     6 | False      | 0.000005s | False      | 0.000001s
     7 | True       | 0.000008s | True       | 0.000063s
     8 | False      | 0.000003s | False      | 0.000001s
     9 | False      | 0.000002s | False      | 0.000007s
    10 | False      | 0.000005s | False      | 0.000001s
    11 | True       | 0.000015s | True       | 0.000076s
    12 | False      | 0.000004s | False      | 0.000001s
    13 | True       | 0.000015s | True       | 0.000089s
    14 | False      | 0.000004s | False      | 0.000001s
    15 | False      | 0.000005s | False      | 0.00000

In [ ]:
import time
import random
import matplotlib.pyplot as plt
import numpy as np

def experiment_execution_time(max_bits=1024, step=32):
    """
    Compare execution time of Miller-Rabin test for different bit sizes
    """
    bits_sizes = list(range(step, max_bits + 1, step))
    times_miller_rabin = []

    print("Bit Size | Execution Time (s)")
    print("-" * 30)

    for bits in bits_sizes:
        print(f"Testing for {bits} bits...", end="")
        n = random.getrandbits(bits) | (1 << (bits - 1)) | 1

        start_time = time.time()
        is_probably_prime(n, k=10)
        end_time = time.time()
        exec_time = end_time - start_time
        times_miller_rabin.append(exec_time)

        print(f" {exec_time:.6f}s")

    plt.figure(figsize=(10, 6))
    plt.plot(bits_sizes, times_miller_rabin, 'bo-', label='Miller-Rabin')

    coeffs = np.polyfit(bits_sizes, times_miller_rabin, 2)
    poly = np.poly1d(coeffs)
    x_trend = np.linspace(min(bits_sizes), max(bits_sizes), 100)
    plt.plot(x_trend, poly(x_trend), 'r--', label=f'Trend: {poly}')

    plt.xlabel('Number of Bits')
    plt.ylabel('Execution Time (seconds)')
    plt.title('Execution Time vs. Bit Size for Miller-Rabin Primality Test')
    plt.legend()
    plt.grid(True)
    plt.savefig('execution_time_comparison.png')
    plt.close()

    print(f"\nResults saved to execution_time_comparison.png")
    return bits_sizes, times_miller_rabin

def experiment_crypto_speed(rsa_bits_sizes=[512, 1024, 2048, 3072, 4096]):
    """
    Demonstrate the impact of primality testing on RSA key generation time
    """
    generation_times = []

    print("\nRSA Key Size | Key Generation Time (s)")
    print("-" * 35)

    for bits in rsa_bits_sizes:
        print(f"Generating {bits}-bit RSA key...", end="")
        start_time = time.time()

        p = generate_large_prime(bits // 2)
        q = generate_large_prime(bits // 2)
        end_time = time.time()
        gen_time = end_time - start_time
        generation_times.append(gen_time)

        print(f" {gen_time:.4f}s")

    plt.figure(figsize=(10, 6))
    plt.bar(rsa_bits_sizes, generation_times, color='green', alpha=0.7)
    plt.xlabel('RSA Key Size (bits)')
    plt.ylabel('Generation Time (seconds)')
    plt.title('Prime Generation Time for RSA Keys of Different Sizes')
    plt.grid(True, axis='y')
    plt.savefig('rsa_key_generation.png')
    plt.close()

    print(f"\nResults saved to rsa_key_generation.png")
    return rsa_bits_sizes, generation_times

def experiment_error_probability(num_tests=50, bit_size=256, witnesses_range=[1, 2, 3, 5, 10, 20, 40]):
    """
    Demonstrate how the number of witnesses affects the error probability
    Analyze consistency across multiple runs
    """
    results = {}

    print("\nNumber of Witnesses | Agreement Rate")
    print("-" * 35)

    for witnesses in witnesses_range:
        print(f"Testing with {witnesses} witnesses...", end="")
        agreements = 0

        for _ in range(num_tests):
            n = generate_large_prime(bit_size, k=40)

            test_result = is_probably_prime(n, k=witnesses)
            if test_result:
                agreements += 1

        agreement_rate = agreements / num_tests
        results[witnesses] = agreement_rate
        print(f" {agreement_rate:.4f}")

    plt.figure(figsize=(10, 6))
    plt.plot(list(results.keys()), list(results.values()), 'ro-')
    plt.xlabel('Number of Witnesses')
    plt.ylabel('Agreement Rate with 40-Witness Test')
    plt.title('Effect of Witness Count on Agreement Rate')
    plt.grid(True)
    plt.savefig('error_probability.png')
    plt.close()

    print(f"\nResults saved to error_probability.png")
    return results

def experiment_large_prime_distribution(bit_sizes=[256, 512, 1024, 2048], num_samples=5):
    """
    Analyze the distribution of large primes
    """
    results = {}

    print("\nBit Size | Avg. Gen Time | # of Digits | First Few Digits | Last Few Digits")
    print("-" * 75)

    for bits in bit_sizes:
        print(f"Generating {num_samples} primes of {bits} bits each...")
        primes = []
        start_time = time.time()

        for _ in range(num_samples):
            prime = generate_large_prime(bits)
            primes.append(prime)

        end_time = time.time()
        avg_time = (end_time - start_time) / num_samples

        results[bits] = {
            'avg_time': avg_time,
            'min_prime': min(primes),
            'max_prime': max(primes),
            'samples': primes
        }

        sample_prime = primes[0]
        prime_str = str(sample_prime)
        num_digits = len(prime_str)

        print(f"{bits:8d} | {avg_time:.6f}s | {num_digits:10d} | {prime_str[:5]}... | ...{prime_str[-5:]}")

    plt.figure(figsize=(10, 6))
    bit_sizes_list = list(results.keys())
    avg_times = [results[bits]['avg_time'] for bits in bit_sizes_list]

    plt.plot(bit_sizes_list, avg_times, 'go-')
    plt.xlabel('Prime Bit Size')
    plt.ylabel('Average Generation Time (seconds)')
    plt.title('Time to Generate Large Primes of Different Sizes')
    plt.grid(True)
    plt.savefig('large_prime_generation_time.png')
    plt.close()

    print(f"\nResults saved to large_prime_generation_time.png")
    return results

def experiment_theoretical_vs_actual():
    """
    Compare theoretical O(log^2 n) performance with actual performance
    """
    bit_sizes = [64, 128, 256, 384, 512, 768, 1024, 1536, 2048]
    times = []

    print("\nBit Size | Execution Time (s)")
    print("-" * 30)

    for bits in bit_sizes:
        print(f"Testing {bits}-bit number...", end="")
        n = random.getrandbits(bits) | (1 << (bits - 1)) | 1

        num_runs = 5
        total_time = 0

        for _ in range(num_runs):
            start_time = time.time()
            is_probably_prime(n, k=10)
            end_time = time.time()
            total_time += (end_time - start_time)

        avg_time = total_time / num_runs
        times.append(avg_time)
        print(f" {avg_time:.6f}s")

    plt.figure(figsize=(10, 6))
    plt.plot(bit_sizes, times, 'bo-', label='Actual Time')

    reference_idx = bit_sizes.index(256)
    k = times[reference_idx] / (256 ** 2)
    theoretical = [k * (b ** 2) for b in bit_sizes]

    plt.plot(bit_sizes, theoretical, 'r--', label='O(log^2 n) Theoretical')

    plt.xlabel('Number of Bits')
    plt.ylabel('Execution Time (seconds)')
    plt.title('Miller-Rabin: Actual vs Theoretical O(log^2 n) Performance')
    plt.legend()
    plt.grid(True)
    plt.savefig('theoretical_vs_actual.png')
    plt.close()

    print(f"\nResults saved to theoretical_vs_actual.png")
    return bit_sizes, times, theoretical

if __name__ == "__main__":
    print("Running primality testing experiments...")

    run_execution_time = True
    run_crypto_speed = True
    run_error_probability = True
    run_distribution = True
    run_theoretical = True

    results = {}

    if run_execution_time:
        print("\nExperiment 1: Execution Time Comparison")
        bits_sizes, times = experiment_execution_time(max_bits=1024, step=64)
        results['execution_time'] = {'bits': bits_sizes, 'times': times}

    if run_crypto_speed:
        print("\nExperiment 2: Cryptography Key Generation Speed")
        key_sizes, gen_times = experiment_crypto_speed([512, 1024, 2048, 3072])
        results['crypto_speed'] = {'key_sizes': key_sizes, 'times': gen_times}

    if run_error_probability:
        print("\nExperiment 3: Error Probability Analysis")
        error_results = experiment_error_probability(num_tests=30)
        results['error_probability'] = error_results

    if run_distribution:
        print("\nExperiment 4: Large Prime Distribution")
        distribution_results = experiment_large_prime_distribution([256, 512, 1024])
        results['distribution'] = distribution_results

    if run_theoretical:
        print("\nExperiment 5: Theoretical vs Actual Performance")
        bit_sizes, actual_times, theoretical_times = experiment_theoretical_vs_actual()
        results['theoretical'] = {
            'bit_sizes': bit_sizes,
            'actual': actual_times,
            'theoretical': theoretical_times
        }

    print("\nAll experiments completed!")

Running primality testing experiments...

Experiment 1: Execution Time Comparison
Bit Size | Execution Time (s)
------------------------------
Testing for 64 bits... 0.000057s
Testing for 128 bits... 0.000085s
Testing for 192 bits... 0.000171s
Testing for 256 bits... 0.000265s
Testing for 320 bits... 0.000416s
Testing for 384 bits... 0.000662s
Testing for 448 bits... 0.000852s
Testing for 512 bits... 0.001238s
Testing for 576 bits... 0.002033s
Testing for 640 bits... 0.002177s
Testing for 704 bits... 0.002538s
Testing for 768 bits... 0.003234s
Testing for 832 bits... 0.003904s
Testing for 896 bits... 0.004636s
Testing for 960 bits... 0.006017s
Testing for 1024 bits... 0.007518s

Results saved to execution_time_comparison.png

Experiment 2: Cryptography Key Generation Speed

RSA Key Size | Key Generation Time (s)
-----------------------------------
Generating 512-bit RSA key... 0.0753s
Generating 1024-bit RSA key... 0.3414s
Generating 2048-bit RSA key... 8.2455s
Generating 3072-bit RSA 

In [ ]:
import time

known_primes = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47]
print("Testing known primes:")
for p in known_primes:
    result = is_probably_prime(p)
    print(f"{p} is prime: {result}")

known_composites = [4, 6, 8, 9, 10, 12, 14, 15, 16, 18, 20, 21]
print("\nTesting known composites:")
for c in known_composites:
    result = is_probably_prime(c)
    print(f"{c} is prime: {result}")

print("\nGenerating and testing primes of increasing bit sizes:")
for bits in [64, 128, 256, 512]:
    print(f"\nGenerating a {bits}-bit prime...")
    start_time = time.time()
    large_prime = generate_large_prime(bits)
    gen_time = time.time() - start_time

    print(f"Generated prime: {large_prime}")
    print(f"Number of digits: {len(str(large_prime))}")
    print(f"Generation time: {gen_time:.6f} seconds")

    start_time = time.time()
    is_prime = is_probably_prime(large_prime)
    verify_time = time.time() - start_time

    print(f"Is prime: {is_prime}")
    print(f"Verification time: {verify_time:.6f} seconds")

Testing known primes:
2 is prime: True
3 is prime: True
5 is prime: True
7 is prime: True
11 is prime: True
13 is prime: True
17 is prime: True
19 is prime: True
23 is prime: True
29 is prime: True
31 is prime: True
37 is prime: True
41 is prime: True
43 is prime: True
47 is prime: True

Testing known composites:
4 is prime: False
6 is prime: False
8 is prime: False
9 is prime: False
10 is prime: False
12 is prime: False
14 is prime: False
15 is prime: False
16 is prime: False
18 is prime: False
20 is prime: False
21 is prime: False

Generating and testing primes of increasing bit sizes:

Generating a 64-bit prime...
Generated prime: 12016217497006807159
Number of digits: 20
Generation time: 0.002095 seconds
Is prime: True
Verification time: 0.002127 seconds

Generating a 128-bit prime...
Generated prime: 329934081340609916755033505646685387841
Number of digits: 39
Generation time: 0.009128 seconds
Is prime: True
Verification time: 0.005368 seconds

Generating a 256-bit prime...
Genera

In [ ]:
import time

bit_sizes = [512, 1024, 2048, 3072, 4096]

print("Testing very large prime numbers:")
print("Bit Size | # of Digits | Generation Time | Verification Time | First 10 digits | Last 10 digits")
print("-" * 90)

for bits in bit_sizes:
    start_time = time.time()
    prime = generate_large_prime(bits)
    gen_time = time.time() - start_time

    start_time = time.time()
    is_prime = is_probably_prime(prime)
    verify_time = time.time() - start_time

    prime_str = str(prime)
    digits = len(prime_str)

    print(f"{bits:8d} | {digits:10d} | {gen_time:14.4f}s | {verify_time:16.4f}s | {prime_str[:10]}... | ...{prime_str[-10:]}")

print("\nGenerating a 4096-bit prime (approximately 1234 decimal digits)...")
start_time = time.time()
huge_prime = generate_large_prime(4096)
gen_time = time.time() - start_time

print(f"Generation time: {gen_time:.4f} seconds")
print(f"Number of decimal digits: {len(str(huge_prime))}")
print(f"First 50 digits: {str(huge_prime)[:50]}...")
print(f"Last 50 digits: ...{str(huge_prime)[-50:]}")

print("\nTesting verification time with different witness counts:")
print("Witnesses | Verification Time (s)")
print("-" * 30)

for k in [5, 10, 20, 40, 80]:
    start_time = time.time()
    is_probably_prime(huge_prime, k=k)
    verify_time = time.time() - start_time

    print(f"{k:9d} | {verify_time:.6f}s")

Testing very large prime numbers:
Bit Size | # of Digits | Generation Time | Verification Time | First 10 digits | Last 10 digits
------------------------------------------------------------------------------------------
     512 |        154 |         0.0602s |           0.0504s | 8882794950... | ...8984606247
    1024 |        309 |         4.5707s |           0.2685s | 1557441110... | ...5874864313
    2048 |        617 |        36.2763s |           1.7947s | 2100416784... | ...9082360727
    3072 |        925 |       363.0766s |           6.1226s | 5402553248... | ...6960342977
    4096 |       1233 |       190.4268s |          12.9778s | 5900146610... | ...3269982673

Generating a 4096-bit prime (approximately 1234 decimal digits)...
Generation time: 50.5399 seconds
Number of decimal digits: 1233
First 50 digits: 69473396944084842876423766306962219923125675810519...
Last 50 digits: ...94391760210699918899434900473091268070935067468369

Testing verification time with different witn